In [ ]:
!pip install dagshub mlflow neuralforecast --quiet

import warnings
from statsmodels.tools.sm_exceptions import ValueWarning

warnings.filterwarnings("ignore", category=ValueWarning)
warnings.filterwarnings("ignore")

print("Done!")

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import mlflow
import mlflow.sklearn
import dagshub
import joblib
import os
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT

pd.set_option("display.max_rows", 100) 

dagshub.init(repo_owner='gnada22', repo_name='ml_final_project', mlflow=True)

mlflow.set_experiment("TFT_Training")

In [ ]:
class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["week"] = (
            X["Date"].dt.to_period("W")
            .rank(method="dense")
            .astype(int) - 1
        )
        X["sin_13"] = np.sin(2 * np.pi * X["week"] / 13)
        X["cos_13"] = np.cos(2 * np.pi * X["week"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["week"] / 23)
        X["cos_23"] = np.cos(2 * np.pi * X["week"] / 23)
        X = X.drop(columns=["Date"])
        return X

date_features = ["week", "sin_13", "cos_13", "sin_23", "cos_23"]
lag_features = []
added_features = date_features + lag_features

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        return X.drop(columns=self.columns, errors="ignore")

class ColumnTransformerWithNames(ColumnTransformer):
    def get_feature_names_out(self, input_features=None):
        return super().get_feature_names_out(input_features)

    def transform(self, X):
        X_transformed = super().transform(X)
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        return res

    def fit_transform(self, X, y=None):
        X_transformed = super().fit_transform(X, y)
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        return res

class MultiIndexKeeper(BaseEstimator, TransformerMixin):
    def __init__(self, index_cols=["Date", "Store", "Dept"]):
        self.index_cols = index_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X.set_index(self.index_cols, drop=False, inplace=True)
        return X

In [ ]:
class TFTRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_chunk_length=52, output_chunk_length=39, epochs=10, batch_size=32):
        self.input_chunk_length = input_chunk_length
        self.output_chunk_length = output_chunk_length
        self.epochs = epochs
        self.batch_size = batch_size

    def fit(self, X, y):
        df = X.copy()
        df["y"] = y.values

        if not isinstance(df.index, pd.MultiIndex):
            raise ValueError("X must have a MultiIndex")

        df = df.reset_index()
        df.rename(columns={"Date": "ds"}, inplace=True)
        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

        self.train_df_ = df[["unique_id", "ds", "y"]].copy()

        model = TFT(
            input_size=self.input_chunk_length,
            h=self.output_chunk_length,
            max_steps=self.epochs * 104,
            batch_size=self.batch_size,
            random_seed=42
        )

        self.nf_ = NeuralForecast(models=[model], freq="W-FRI")
        self.nf_.fit(df=self.train_df_)
        return self

    def predict(self, X):
        df = X.reset_index()
        df.rename(columns={"Date": "ds"}, inplace=True)
        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

        forecast_df = self.nf_.predict()
        forecast_df = forecast_df.rename(columns={"TFT": "y_hat"})

        merged = df.merge(forecast_df, on=["unique_id", "ds"], how="left")
        preds = pd.Series(data=merged["y_hat"].fillna(0).values, index=X.index)

        return preds.to_numpy()

In [ ]:
categorical = ["Store", "Dept", "Type", "IsHoliday"]
numerical = ["Temperature", "Fuel_Price", "CPI", "Unemployment",
             "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]
engineered = added_features

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numerical_transformer = SimpleImputer(strategy="mean")

preprocessor = ColumnTransformerWithNames(
    transformers=[
        ("num", numerical_transformer, numerical + engineered),
        ("cat", categorical_transformer, categorical),
    ],
    sparse_threshold=0.0
)

pipeline = Pipeline([
    ("index_keeper", MultiIndexKeeper()),
    ("date_features", DateFeatureCreator()),
    ("preprocessor", preprocessor),
    ("column_dropper", ColumnDropper(columns=['Date', 'Store', 'Dept'])),
    ("patchtst", TFTRegressor(epochs=25))
])

In [ ]:
TRAIN_ON_ENTIRE_DATA = True
print(TRAIN_ON_ENTIRE_DATA)

In [ ]:
# Load and merge data
df_train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
df_features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])
df_stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")

df = df_train.merge(df_features, on=["Store", "Date", "IsHoliday"], how="left")
df = df.merge(df_stores, on="Store", how="left")
df = df.sort_values(by=['Date', 'Store', 'Dept'])

with mlflow.start_run(run_name="Feature_Engineering"):
    mlflow.log_param("features_added", added_features)

y = df["Weekly_Sales"]
X = df.drop(columns=["Weekly_Sales"])

split_idx = int(len(X) * 0.8)
X_train, X_val = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_val = y.iloc[:split_idx], y.iloc[split_idx:]

if TRAIN_ON_ENTIRE_DATA:
    X_train = X.copy()
    y_train = y.copy()

# print(X_train)
# print(X_val)

In [ ]:
with mlflow.start_run(run_name="Training"):
    pipeline.fit(X_train, y_train)
    best_model = pipeline

    if not TRAIN_ON_ENTIRE_DATA:
        preds = pipeline.predict(X_val)
        mae = mean_absolute_error(y_val, preds)
        weights = X_val["IsHoliday"].apply(lambda x: 5 if x else 1)
        wmae = (weights * np.abs(y_val - preds)).sum() / weights.sum()
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("WMAE", wmae)
        # print("y:\n", y)
        # print("preds:\n", preds)

    model_path = "model.pkl"
    joblib.dump(best_model, model_path)
    mlflow.log_artifact(model_path)
    
    # mlflow.sklearn.log_model(best_model, artifact_path="model")